In [3]:
# Just run this block. Please do not modify the following code.
import math
import time
import io
import numpy as np
import csv
from IPython.display import Image

# Pytorch package
import torch
import torch.nn as nn
import torch.optim as optim

# Torchtest package
# import torchtext
# from torchtext.datasets import Multi30k
# from torch.utils.data import DataLoader
# from torchtext.data.utils import get_tokenizer
# from collections import Counter
# from torchtext.vocab import vocab
# from torchtext.utils import download_from_url, extract_archive
# from torch.nn.utils.rnn import pad_sequence
from torch.optim import Adam

# Tqdm progress bar
from tqdm import tqdm_notebook, tqdm

# Code provide to you for training and evaluation

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

torch.seed

<function torch.random.seed() -> int>

In [4]:
# Check device availability
device = ''
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
device = torch.device(device)
print("You are using device: %s" % device)

You are using device: mps


In [5]:
encoder_output = torch.rand((5, 320, 20)).to(device) # N, input_size, T

In [6]:
from Decoder import Decoder
from PostNet import PostNet

In [7]:
decoder = Decoder(lstm_bidirectional=True).to(device)
decoder_output = decoder.forward(encoder_output)
print(decoder_output.shape) # N, output_size, T
print(decoder_output)

torch.Size([5, 80, 20])
tensor([[[ 3.6825e-03,  4.4606e-02,  1.0958e-01,  ...,  1.3586e-02,
           4.8501e-03,  3.3209e-02],
         [ 3.1870e-02,  1.7513e-03,  2.4678e-02,  ..., -7.8045e-03,
           6.1482e-03, -2.8795e-02],
         [ 5.9142e-02,  1.0487e-01,  9.0599e-02,  ...,  1.3067e-03,
          -1.0156e-03,  8.4302e-03],
         ...,
         [-5.2882e-02, -9.0974e-02, -6.2130e-02,  ...,  8.5704e-03,
          -2.2412e-02, -1.1856e-04],
         [-2.5380e-02, -2.9963e-02,  1.3823e-02,  ..., -9.6502e-03,
           2.2265e-02,  1.1504e-02],
         [-5.4349e-02, -4.3660e-02,  8.8643e-03,  ..., -2.3915e-02,
           3.9987e-02,  1.7217e-03]],

        [[ 5.8105e-02,  7.0889e-02,  1.1113e-01,  ...,  7.1480e-02,
           3.6759e-02,  6.8150e-02],
         [ 5.5411e-02,  4.0619e-02,  7.2729e-02,  ..., -4.1806e-02,
           1.1533e-02,  2.1892e-02],
         [ 8.4126e-02,  1.0570e-01,  1.1730e-01,  ...,  7.8362e-02,
           8.4697e-02,  4.0918e-02],
         ...,
 

In [8]:
postnet = PostNet().to(device)
postnet_output = postnet.forward(decoder_output)
print(postnet_output.shape) # (N, output size, T)
print(postnet_output)

torch.Size([5, 80, 20])
tensor([[[-0.1417, -0.2846,  0.2124,  ...,  0.6432,  0.3604,  0.2329],
         [-0.4447, -0.4212, -0.2686,  ...,  0.3764,  0.1740,  0.0898],
         [ 0.1103, -0.1607,  0.1748,  ..., -0.0634, -0.9434, -0.4235],
         ...,
         [ 0.0169,  0.3285, -0.3866,  ...,  0.3348,  0.3624,  0.0397],
         [ 0.1127, -0.5509,  0.6533,  ..., -0.2584,  0.1697, -0.1248],
         [ 0.1750, -0.0609, -0.8094,  ..., -0.7462, -0.1410, -0.0392]],

        [[-0.4792, -0.5449, -0.6200,  ...,  0.2085, -0.0342, -0.1272],
         [-0.0577, -0.4932, -0.9531,  ...,  0.5104,  0.2924, -0.0315],
         [ 0.1564, -0.0919,  0.0179,  ...,  0.0664,  0.0717, -0.2495],
         ...,
         [ 0.0839, -0.0158, -0.0715,  ...,  0.1605, -0.1189,  0.4460],
         [-0.1325,  0.0530,  0.3684,  ...,  0.1296, -0.5494, -0.2993],
         [ 0.9531,  0.7085,  0.3954,  ..., -0.1096,  0.0414, -0.0203]],

        [[ 0.3447,  0.4252, -0.2401,  ...,  0.4469,  0.2567, -0.0585],
         [ 0.2721, -0

In [9]:
# Loss Test
from utils import criterion
#
input = torch.rand((5, 80, 1)).to(device)

print(input.shape)
print(encoder_output.shape)
print(decoder_output.shape)
print(postnet_output.shape)

criterion(input=input, encoder_ouput=encoder_output, decoder_output=decoder_output, postnet_output=postnet_output, encoded_postnet_output=encoder_output)

torch.Size([5, 80, 1])
torch.Size([5, 320, 20])
torch.Size([5, 80, 20])
torch.Size([5, 80, 20])


/opt/homebrew/anaconda3/envs/cs7643-finalproject-v2/lib/python3.10/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([5, 80, 20])) that is different to the input size (torch.Size([5, 80, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(0.7716, device='mps:0', grad_fn=<AddBackward0>)

# Training, evaluation, and plotting loss curves

In [1]:
# Code provide to you for training and evaluation
from utils import train
from utils import evaluate
from utils import plot_curves
from Model import Model

def train_and_plot(model, optimizer, scheduler, criterion, train_loader, valid_loader, filename, epochs, device='cpu'):
    train_loss_history = []
    valid_loss_history = []

    for epoch_idx in range(epochs):
        print("-----------------------------------")
        print("Epoch %d" % (epoch_idx+1))
        print("-----------------------------------")

        train_loss, avg_train_loss = train(model, train_loader, optimizer, criterion, device=device)
        scheduler.step(train_loss)

        val_loss, avg_val_loss = evaluate(model, valid_loader, criterion, device=device)

        train_loss_history.append(avg_train_loss)
        valid_loss_history.append(avg_val_loss)

        print("Training Loss: %.4f. Validation Loss: %.4f." % (avg_train_loss, avg_val_loss))

    plot_curves(train_loss_history, valid_loss_history, filename)


In [ ]:

# Example usage:
model = Model()
train_loader = DataLoader(your_train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(your_valid_dataset, batch_size=32, shuffle=False)
optimizer = Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
criterion = criterion(input=input, encoder_ouput=encoder_output, decoder_output=decoder_output, postnet_output=postnet_output, encoded_postnet_output=encoder_output)
filename = "training_loss_curves"
epochs = 10
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_and_plot(model, optimizer, scheduler, criterion, train_loader, valid_loader, filename, epochs, device)